This script is necessary to identify the structures needed to compute the Climate Shelter Index (CSI), in particular:
- Surface Area of Green Spaces
- Presence of drinking fountains
- Presence of picnic tables and benches (rest area)

In addition the data about green area are combined with data about parks and gardens in Bologna

### 0. Libraries

In [1]:
# libraries
import os
import geopandas as gpd
import shapely  #1.8.5 version 
from shapely.geometry import Polygon
from shapely.geometry import Point
import matplotlib
import mapclassify
import rasterio
import numpy as np # 1.23.5 version
import pandas as pd
import urllib.request
import pyrosm

c:\Users\elima\anaconda3\envs\geoss\Lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


### 1. Surface Area 
Calculate the surface area of each green space in the municipality of Bologna

In [2]:
# Load the shapefile of green area 
ar_vrd = gpd.read_file(r"D:\Bologna_DBSN\Bologna_dbsn_selected\ar_vrd.shp")

In [3]:
# check the current CRS
ar_vrd.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
target_crs = "EPSG:32633"
# Reproject the GeoDataFrame to the target CRS
ar_vrd = ar_vrd.to_crs(target_crs)

In [6]:
# Calculate Surface Area in meters an hectares
ar_vrd['Area_SqMeters'] = ar_vrd.geometry.area
ar_vrd['Area_Ha'] = ar_vrd['Area_SqMeters'] / 10000

In [8]:
# Reproject the GeoDataFrame to the orignal CRS
ar_vrd= ar_vrd.to_crs(epsg=4326)

# Save the GeoDataFrame with the surface area as a shapefile
ar_vrd.to_file(r'D:\Climate_Shelter_Index\ar_vrd_surface.shp')

C:\Users\elima\AppData\Local\Temp\ipykernel_1344\3525987562.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ar_vrd.to_file(r'D:\Climate_Shelter_Index\ar_vrd_surface.shp')


### 2. Drinking fountains
Load data about drinking fountains

In [ ]:
url_download_bologna_pbf = 'https://osmit-estratti.wmcloud.org/dati/poly/comuni/pbf/037006_Bologna.osm.pbf'
urllib.request.urlretrieve(url_download_bologna_pbf ,"bologna_osm.pbf")
osm = pyrosm.OSM("bologna_osm.pbf") 

In [ ]:
# Identify drinking fountains using some filters
custom_filter = {'amenity': ['drinking_water']}
drinking_fountain = osm.get_pois(custom_filter=custom_filter)

In [ ]:
# Create a GeoDataFrame for drinking fountains
drinking_fountain_gdf = gpd.GeoDataFrame(drinking_fountain, geometry=[Point(x, y) for x, y in zip(drinking_fountain["lon"], drinking_fountain["lat"])])

In [ ]:
# Save the GeoDataFrame as a shapefile
drinking_fountain_gdf.to_file('D:\Climate_Shelter_Index\drinking_fountains.shp')

#### Green areas + Drinking fountains
Check if there are drinking fountains in the green areas

In [15]:
# Load the shapefiles
green_area = gpd.read_file(r'D:\Climate_Shelter_Index\ar_vrd_surface.shp')
fountains = gpd.read_file(r"D:\Climate_Shelter_Index\drinking_fountains.shp")

In [17]:
# Perform a spatial join to check if each park contains a fountain
ga_with_fountains = gpd.sjoin(green_area, fountains, how="left", op="intersects")

# Create a new column 'drinking_fountain' indicating if there is a fountain
ga_with_fountains["drinking_fountain"] = ga_with_fountains["index_right"].notnull().replace({True: "yes", False: "no"})

c:\Users\elima\anaconda3\envs\geoss\Lib\site-packages\IPython\core\interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [18]:
# Create a new column 'id_fountain' with the ID of the drinking fountain if present
ga_with_fountains["id_fountain"] = ga_with_fountains["id"].fillna("N/A")

In [14]:
# Check all the columns
ga_with_fountains.columns

Index(['OBJECTID', 'ar_vrd_ty', 'scril', 'classid', 'meta_ist', 'shape_Leng',
       'shape_Area', 'Area_SqMet', 'Area_Ha', 'geometry', 'index_right',
       'version', 'id', 'lat', 'tags', 'changeset', 'timestamp', 'lon', 'name',
       'amenity', 'fountain', 'source', 'osm_type', 'drinking_fountain',
       'id_fountain'],
      dtype='object')

In [20]:
# Select the columns you want to keep in the final GeoDataFrame
final_columns = ["OBJECTID", "classid", "shape_Leng", "shape_Area", "Area_SqMet", "Area_Ha", "geometry", 'amenity',
                 'drinking_fountain' ,'id_fountain']


In [21]:
# Create a new GeoDataFrame with the selected columns
final_gdf = ga_with_fountains[final_columns]

final_gdf["id_fountain"] =  final_gdf["id_fountain"].apply(lambda x: str(x).replace('.0', '') if x != 'N/A' else x)

final_gdf[final_gdf['drinking_fountain']=='yes'].head(3)


c:\Users\elima\anaconda3\envs\geoss\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,OBJECTID,classid,shape_Leng,shape_Area,Area_SqMet,Area_Ha,geometry,amenity,drinking_fountain,id_fountain
10684,10685.0,ee361836-dfdd-4f03-b723-2600d9435523,7807.121351,48816.948067,49024.110818,4.902411,"POLYGON Z ((11.32203 44.49533 0.00000, 11.3218...",drinking_water,yes,9083886142
10693,10694.0,998e0bdf-a3ab-498a-8b1f-09fc6016460f,5774.082337,44318.074700,44507.603047,4.450760,"POLYGON Z ((11.28510 44.51163 0.00000, 11.2850...",drinking_water,yes,8130019758
10810,10811.0,2394871a-6e60-47de-adf1-e0b950cccaa5,5688.140142,75708.860246,76027.876725,7.602788,"POLYGON Z ((11.35577 44.46183 0.00000, 11.3556...",drinking_water,yes,7242733300


In [22]:
# Rename the column
final_gdf.rename(columns={"Area_SqMet": "Area_SqM",
                          "drinking_fountain":"d_fountain",
                          "id_fountain":"id_df"
                          }, 
                 inplace=True)

C:\Users\elima\AppData\Local\Temp\ipykernel_10716\1714682596.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_gdf.rename(columns={"Area_SqMet": "Area_SqM",


In [23]:
# Save the final GeoDataFrame as a shapefile
final_gdf.to_file(r"D:\Climate_Shelter_Index\ar_vrd_surface_drinkingfount.shp")


### 3. Picnic tables
Load data about picnic tables

In [9]:
# Identify picnic tables using some filters
custom_filter = {'leisure': ['picnic_table']}
picnic_table = osm.get_pois(custom_filter=custom_filter)

c:\Users\elima\anaconda3\envs\geoss\Lib\site-packages\pyrosm\pyrosm.py:576: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  gdf = get_poi_data(


In [12]:
# Create a GeoDataFrame for picnic tables
picnic_table_gdf = gpd.GeoDataFrame(picnic_table, geometry=[Point(x, y) for x, y in zip(picnic_table["lon"], picnic_table["lat"])])

In [15]:
# Save the GeoDataFrame as a shapefile
picnic_table_gdf.to_file('D:\Climate_Shelter_Index\picnic_tables.shp')

#### Green areas + Drinking fountains + Picnic tables
Check if there are picnic tables in the green areas

In [24]:
# Load the shapefiles
ar_vrd_surface_drinkingfount = gpd.read_file(r"D:\Climate_Shelter_Index\ar_vrd_surface_drinkingfount.shp")
picnic_tables = gpd.read_file(r'D:\Climate_Shelter_Index\picnic_tables.shp')

In [26]:
# Perform a spatial join to check if each park contains a picnic table
ga_pt = gpd.sjoin(ar_vrd_surface_drinkingfount, picnic_tables, how="left", op="intersects")

# Create a new column 'picnic_table' indicating if there is a picnic table
ga_pt["picnic_table"] = ga_pt["index_right"].notnull().replace({True: "yes", False: "no"})

c:\Users\elima\anaconda3\envs\geoss\Lib\site-packages\IPython\core\interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [28]:
# Create a new column 'id_pt' with the ID of the picnic table if present
ga_pt["id_pt"] = ga_pt["id"].fillna("N/A")

In [35]:
final_columns = ['OBJECTID', 'classid', 'shape_Leng', 'shape_Area', 'Area_SqM',
        'Area_Ha', 'amenity', 'd_fountain', 'id_df', 'geometry','picnic_table', 'id_pt']
# Create a new GeoDataFrame with the selected columns
final_gdf2 = ga_pt[final_columns]

final_gdf2['id_pt'] = final_gdf2['id_pt'].apply(lambda x: str(x).replace('.0', '') if x != 'N/A' else x)


c:\Users\elima\anaconda3\envs\geoss\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [37]:
# Save the final GeoDataFrame as a shapefile
final_gdf2.to_file(r"D:\Climate_Shelter_Index\ar_vrd_s_df_pt.shp")

C:\Users\elima\AppData\Local\Temp\ipykernel_10716\2292307633.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  final_gdf2.to_file(r"D:\Climate_Shelter_Index\ar_vrd_s_df_pt.shp")


### 4. Benches
Load data about benches

In [4]:
# Identify benches using some filters
custom_filter = {'amenity': ['bench']}
benches = osm.get_pois(custom_filter=custom_filter)

c:\Users\elima\anaconda3\envs\geoss\Lib\site-packages\pyrosm\pyrosm.py:576: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  gdf = get_poi_data(


In [7]:
# Create a GeoDataFrame for benches
benches_gdf = gpd.GeoDataFrame(benches, geometry=[Point(x, y) for x, y in zip(benches["lon"], benches["lat"])])

In [9]:
# Save the GeoDataFrame as a shapefile
benches_gdf.to_file(r'D:\Climate_Shelter_Index\benches.shp')

#### Green areas + Drinking fountains + Picnic tables + Benches
Check if there are benches in the green areas

In [10]:
# Load shapefiles
ar_vrd_s_df_pt = gpd.read_file(r"D:\Climate_Shelter_Index\ar_vrd_s_df_pt.shp")
benches = gpd.read_file(r'D:\Climate_Shelter_Index\benches.shp')

In [12]:
# Perform a spatial join to check if each park contains benches
ga_benches = gpd.sjoin(ar_vrd_s_df_pt, benches, how="left", op="intersects")

# Create a new column 'benches' indicating if there is a bench
ga_benches["benches"] = ga_benches["index_right"].notnull().replace({True: "yes", False: "no"})

c:\Users\elima\anaconda3\envs\geoss\Lib\site-packages\IPython\core\interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [13]:
# Create a new column 'id_bench' with the ID of the bench if present
ga_benches["id_bench"] = ga_benches["id"].fillna("N/A")

In [18]:
ga_benches.columns

Index(['OBJECTID', 'classid', 'shape_Leng', 'shape_Area', 'Area_SqM',
       'Area_Ha', 'amenity_left', 'd_fountain', 'id_df', 'picnic_tab', 'id_pt',
       'geometry', 'index_right', 'id', 'version', 'lat', 'changeset', 'lon',
       'timestamp', 'tags', 'amenity_right', 'osm_type', 'benches',
       'id_bench'],
      dtype='object')

In [24]:
final_columns = ['OBJECTID', 'classid', 'shape_Leng', 'shape_Area', 'Area_SqM',
        'Area_Ha',  'd_fountain', 'id_df', 'geometry','picnic_tab', 'id_pt',
        'benches', 'id_bench']
# Create a new GeoDataFrame with the selected columns
final_gdf3 = ga_benches[final_columns]

final_gdf3['id_bench'] = final_gdf3['id_bench'].apply(lambda x: str(x).replace('.0', '') if x != 'N/A' else x)


c:\Users\elima\anaconda3\envs\geoss\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [29]:
final_gdf3['OBJECTID'] = final_gdf3['OBJECTID'].apply(lambda x: str(x).replace('.0', '') if x != 'N/A' else x)

c:\Users\elima\anaconda3\envs\geoss\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [31]:
# Save the final GeoDataFrame as a shapefile
final_gdf3.to_file(r"D:\Climate_Shelter_Index\ar_vrd_s_df_pt_b.shp")

### 5. Park names
Load data about garden and park in Bologna (with their names)

In [ ]:
#Load shapefile
ar_vrd_s_df_pt_b = gpd.read_file(r"D:\Climate_Shelter_Index\ar_vrd_s_df_pt_b.shp")
park_names = gpd.read_file(r"D:\Climate_Shelter_Index\carta-tecnica-comunale-toponimi-parchi-e-giardini\carta-tecnica-comunale-toponimi-parchi-e-giardini.shp")

In [4]:
# Perform a spatial join 
final_gdf4 = gpd.sjoin(ar_vrd_s_df_pt_b, park_names, how="left", op="intersects")

c:\Users\elima\anaconda3\envs\geoss\Lib\site-packages\IPython\core\interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [6]:
final_gdf4['index_right'] = final_gdf4['index_right'].apply(lambda x: str(x).replace('.0', '') if x != 'N/A' else x)

In [12]:
# Rename the column
final_gdf4.rename(columns={"index_right": "id_park",}, 
                 inplace=True)

In [13]:
# Save the final GeoDataFrame as a shapefile
final_gdf4.to_file(r"D:\Climate_Shelter_Index\ar_vrd_s_df_pt_b_names.shp")